Regressions between ERA5 (dif. variables) and Cloud Index (CI) in Corrientes Argentina

Las anomalias de CI se generaron asi:

* Calculo el CI mensuales para los meses con menos del 20% de datos faltantes
* Calculo las anomalias de CI mensuales (le resto a las medias mensuales la CI climatologica de cada mes calculada sobre el periodo comun con el satelite: fecha_inicio_climatologia_str = "1983-12-15"
fecha_final_climatologia_str = "2016-11-15")
* Todo se guarda en csvs: cada columna es una estacion. En: "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2021/nubosidad/paper"
TCC_monthly_CI_SMN_anomaly.csv

ERA5 is for southamerica

In [40]:
import os
import pandas as pd 
import numpy as np
import datetime 

import iris
from iris.analysis import MEAN
from iris.analysis.stats import pearsonr
import iris.coord_categorisation

import scipy
from scipy.stats import theilslopes #theil-sen trends

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
# directories
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

#si estoy trabajando desde el servidor vegeta
ruta_a_archivos_ERA5_mensual = "/pikachu/datos/nadia.testani/Doctorado/datos/ERA5/mon/SAm" #ruta donde estan los archivos con datos de ERA5
ruta_a_archivos_CI = "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2021/nubosidad/paper"
ruta_a_resultados = "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2024" #ruta donde se guardan los resultados del analisis de nubosidad

Current working directory: /home/nadia.testani/Doctorado/DoctoradoCodigos/codigos2024


In [3]:
#Cloud Index

CI_anomaly = pd.read_csv(f"{ruta_a_archivos_CI}/TCC_monthly_CI_SMN_anomaly.csv")
#from 1961-01-01 until 2021-11-01

#set dates as index
CI_anomaly = CI_anomaly.set_index(CI_anomaly["Unnamed: 0"]) 
CI_anomaly = CI_anomaly.drop(["Unnamed: 0"], axis = 1)
CI_anomaly.index = pd.DatetimeIndex(CI_anomaly.index)

#filter theil-sen trend
def calculate_theilsen_slope(y):
    slope, intercept, _, _ = theilslopes(y, range(len(y)), alpha=0.95)
    return slope, intercept

CI_anomaly_no_trends = CI_anomaly.copy()
for column in CI_anomaly_no_trends: #iterate throw every station
    serie = CI_anomaly_no_trends[column]
    valores = serie.values[np.isfinite(serie.values)]
    trend = calculate_theilsen_slope(valores)[0] * np.arange(len(serie)) + calculate_theilsen_slope(valores)[1]
    CI_anomaly_no_trends[column] = serie - trend

#regional mean value
CI_anomaly_no_trends_regional_mean = CI_anomaly_no_trends.mean(axis=1)

In [20]:
#ERA5
#ERA5_netcdf = iris.load_cube(f"{ruta_a_archivos_ERA5_mensual}/ERA5_zg_SA.nc") #geopotencial
variable_name = 't2m'
ERA5_netcdf = iris.load_cube(f"{ruta_a_archivos_ERA5_mensual}/new_data.nc", variable_name) #geopotencial
ERA5_netcdf

<iris 'Cube' of 2 metre temperature / (K) (time: 768; expver: 2; latitude: 301; longitude: 217)>

In [27]:
ERA5_netcdf_1 = ERA5_netcdf[:,0,:,:] #select expver=1

In [55]:
#Select times = times of CI
time_coord = ERA5_netcdf_1.coord("time") 
time_axis = time_coord.units.num2date(time_coord.core_points())
timei = datetime.datetime(1961, 1, 1, 0, 0) 
timef = datetime.datetime(2021, 12, 1, 0, 0) 

# Create a constraint based on the chosen datetime
time_constraint = iris.Constraint(time=lambda cell: timei <= cell.point <= timef)

# Apply the constraint to slice the cube
ERA5_netcdf_1_datesCI = ERA5_netcdf_1.extract(time_constraint)

time_coord = ERA5_netcdf_1_datesCI.coord("time")
time_axis = time_coord.units.num2date(time_coord.core_points())


In [56]:
#Select times = times of climatology
#fecha_inicio_climatologia_str = "1983-12-15"
#fecha_final_climatologia_str = "2016-11-15"

time_coord = ERA5_netcdf_1.coord("time") 
time_axis = time_coord.units.num2date(time_coord.core_points())
timei = datetime.datetime(1983, 12, 1, 0, 0) 
timef = datetime.datetime(2016, 11, 1, 0, 0) 

# Create a constraint based on the chosen datetime
time_constraint = iris.Constraint(time=lambda cell: timei <= cell.point <= timef)

# Apply the constraint to slice the cube
ERA5_netcdf_1_datesClimatology = ERA5_netcdf_1.extract(time_constraint)

time_coord = ERA5_netcdf_1_datesClimatology.coord("time")
time_axis = time_coord.units.num2date(time_coord.core_points())

#mean values
# calculate monthly mean
# Categorize the time coordinate by month
iris.coord_categorisation.add_month(ERA5_netcdf_1_datesClimatology, 'time', name='month')

# Calculate the monthly mean for each grid point
monthly_means = ERA5_netcdf_1_datesClimatology.aggregated_by('month', iris.analysis.MEAN)

In [61]:
#theil-sen trend takes LONG
def calculate_theilsen_slope(y):
    slope, intercept, _, _ = theilslopes(y, range(len(y)), alpha=0.95)
    return slope, intercept
# Apply the Theil-Sen slope calculation to each grid point
trend_cube = ERA5_netcdf_1_datesCI[0:2,:,:].copy()
trend_cube.data = np.apply_along_axis(calculate_theilsen_slope, axis=0, arr=ERA5_netcdf_1_datesCI.data)

#filter theil-sen trend
def filter_theilsen_slope(y):
    #y: array 1D
    trend = calculate_theilsen_slope(y)[0] * np.arange(len(y)) + calculate_theilsen_slope(y)[1]
    filter_data = y - trend
    return filter_data
data_trend_filtered_datesCI = ERA5_netcdf_1_datesCI[:,:,:].copy()
data_trend_filtered_datesCI.data = np.apply_along_axis(filter_theilsen_slope, axis=0, arr=ERA5_netcdf_1_datesCI.data)


In [62]:
# Filter monthly means --> anomalies
monthly_means_array = np.repeat(monthly_means.data, len(data_trend_filtered_datesCI.data[:,1,1])/12, axis=0)
ERA5_trend_filtered_anomalies = data_trend_filtered_datesCI.data - monthly_means_array

#I have to delete the last december to calculate the regression

 Detrend xarray with theil-sen FAST WAY

 Based on:

 https://docs.esmvaltool.org/projects/ESMValCore/en/latest/api/esmvalcore.preprocessor.html#esmvalcore.preprocessor.detrend
 https://docs.esmvaltool.org/projects/ESMValCore/en/latest/_modules/esmvalcore/preprocessor/_detrend.html#detrend

Create my own detrend using theil-sen. Use this as a model: 
https://github.com/scipy/scipy/blob/906e6cd9bb022e78a07b34636bee64813ce56afa/scipy/signal/_signaltools.py#L3521

In [109]:
import numpy as np
from scipy import signal, linalg

def detrend_theil_sen(data, axis=-1, type='linear', bp=0, overwrite_data=False):
    """
    Remove linear trend along axis from data using Theil-Sen slopes.

    Parameters
    ----------
    data : array_like
        The input data.
    axis : int, optional
        The axis along which to detrend the data. By default, this is the
        last axis (-1).
    type : {'linear', 'constant'}, optional
        The type of detrending. If ``type == 'linear'`` (default),
        the result of a Theil-Sen fit to `data` is subtracted from `data`.
        If ``type == 'constant'``, only the mean of `data` is subtracted.
    bp : array_like of ints, optional
        A sequence of break points. If given, an individual Theil-Sen fit is
        performed for each part of `data` between two break points.
        Break points are specified as indices into `data`. This parameter
        only has an effect when ``type == 'linear'``.
    overwrite_data : bool, optional
        If True, perform in place detrending and avoid a copy. Default is False

    Returns
    -------
    ret : ndarray
        The detrended input data.

    Examples
    --------
    >>> import numpy as np
    >>> from scipy import signal
    >>> rng = np.random.default_rng()
    >>> npoints = 1000
    >>> noise = rng.standard_normal(npoints)
    >>> x = 3 + 2*np.linspace(0, 1, npoints) + noise
    >>> (detrend_theil_sen(x) - noise).max()
    0.06  # random

    """
    if type not in ['linear', 'l', 'constant', 'c']:
        raise ValueError("Trend type must be 'linear' or 'constant'.")
    data = np.asarray(data)
    dtype = data.dtype.char
    if dtype not in 'dfDF':
        dtype = 'd'
    if type in ['constant', 'c']:
        ret = data - np.mean(data, axis, keepdims=True)
        return ret
    else:
        dshape = data.shape
        N = dshape[axis]
        bp = np.sort(np.unique(np.concatenate(np.atleast_1d(0, bp, N))))
        if np.any(bp > N):
            raise ValueError("Breakpoints must be less than the length "
                             "of data along the given axis.")

        # Restructure data so that the axis is along the first dimension
        # and all other dimensions are collapsed into the second dimension
        rnk = len(dshape)
        if axis < 0:
            axis = axis + rnk
        newdata = np.moveaxis(data, axis, 0)
        newdata_shape = newdata.shape
        newdata = newdata.reshape(N, -1)

        if not overwrite_data:
            newdata = newdata.copy()  # make sure we have a copy
        if newdata.dtype.char not in 'dfDF':
            newdata = newdata.astype(dtype)

        # Find Theil-Sen fit and remove it for each piece
        for m in range(len(bp) - 1):
            sl = range(bp[m], bp[m + 1])  # Convert slice to range
            x = np.arange(1, len(sl) + 1, dtype=dtype) / len(sl)
            y = newdata[sl]
            slopes = np.diff(y) / np.diff(x)
            median_slope = np.median(slopes)
            newdata[sl] = y - median_slope * x

        # Put data back in the original shape.
        newdata = newdata.reshape(newdata_shape)
        ret = np.moveaxis(newdata, 0, axis)
        return ret

In [110]:
"""Preprocessor functions that remove trends from the data."""
import logging

import dask.array as da
import scipy.signal

def detrend_theil_sen_cube(cube, dimension='time', method='linear'):
    """
    Detrend data along a given dimension.

    Parameters
    ----------
    cube: iris.cube.Cube
        input cube.
    dimension: str
        Dimension to detrend
    method: str
        Method to detrend. Available: linear, constant. See documentation of
        'scipy.signal.detrend' for details

    Returns
    -------
    iris.cube.Cube
        Detrended cube
    """
    coord = cube.coord(dimension)
    axis = cube.coord_dims(coord)[0]
    detrended = da.apply_along_axis(
        detrend_theil_sen,
        axis=axis,
        arr=cube.lazy_data(),
        type=method,
        shape=(cube.shape[axis],)
    )
    return cube.copy(detrended)

In [111]:
detrend_theil_sen_cube(ERA5_netcdf_1, dimension='time', method='linear')

/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


<iris 'Cube' of 2 metre temperature / (K) (time: 768; latitude: 301; longitude: 217)>